# Environment 1

In [1]:
import pandas as pd
import numpy as np
import requests
from time import sleep
from keras.layers import Input, Embedding, Flatten, Dot
from keras.models import Model

Using TensorFlow backend.


This environment encode the classic explicit feedback setting with user, item and rating.

In [2]:
# require:
# user id
# predicted_score for previous (user, item)

# return:
# rating, next_user, next_item
r = requests.get(url='http://52.47.62.31/predict', 
                 params={'user_id': '5PZFMN4UIV66SBO6A1KE',
                         'predicted_score': 0.1})

data = r.json()

In [3]:
data

{'next_item': 121, 'next_user': 35, 'rating': 2}

In [4]:
user_id = '0EFZ3W3H6IQRD9DDSPS2'
base_url = 'http://52.47.62.31'
url_reset = base_url + '/reset' # Restart the environment with new random values.
url_predict = base_url + '/predict'
params = {'user_id': user_id}

In [5]:
# json file
r = requests.get(url=url_reset, params=params)
data = r.json()

In [6]:
data.keys()

dict_keys(['item_history', 'nb_items', 'nb_users', 'next_item', 'next_user', 'rating_history', 'user_history'])

In [7]:
nb_items = data['nb_items']
nb_users = data['nb_users']
user_history = data['user_history']
item_history = data['item_history']
rating_history = data['rating_history']

next_user = data['next_user']
next_item = data['next_item']

# First method using simularity

In [8]:
# create dataframe
df = pd.DataFrame(user_history, columns=['user'])
df['item'] = item_history
df['rating'] = rating_history
df.head(10)

,user,item,rating
0,32,216,3
1,97,70,5
2,50,106,1
3,53,254,2
4,51,173,1
5,1,236,4
6,31,276,3
7,95,154,2
8,63,50,5
9,72,98,3


In [9]:
# matrix
mat = df.pivot_table(values=['rating'], index=['user'], columns=['item'])

ratings = mat.fillna(0).values
ratings

array([[3., 1., 0., ..., 2., 3., 0.],
       [0., 1., 0., ..., 0., 4., 0.],
       [0., 1., 5., ..., 5., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 2., 2., ..., 0., 0., 5.]])

In [10]:
num_non_zero = np.count_nonzero(ratings)
print(num_non_zero/ratings.shape[0]/ratings.shape[1])

0.3333333333333333


In [11]:
def similarity(ratings):
    
    r_user = (ratings>0).sum(axis=1)  
    
    m_user = np.divide(ratings.sum(axis=1) , r_user, where=r_user!=0)
    
    ratings_ctr = ratings.T - ((ratings.T!=0) * m_user)
    ratings_ctr = ratings_ctr.T

    sim = ratings_ctr.dot(ratings_ctr.T)
    
    norms = np.array([np.sqrt(np.diagonal(sim))])
    sim = sim / norms / norms.T  

    return sim

In [12]:
def phi(x):
    return np.maximum(x, 0)

In [13]:
sim = similarity(ratings)
sim = np.nan_to_num(sim)
numerator = phi(sim).dot(ratings)
denominator = phi(sim).dot(ratings > 0)
pred_ratings = np.divide(numerator,denominator,where = denominator!=0)

In [14]:
pred_ratings

array([[3.38002595, 1.1047499 , 4.05589672, ..., 3.57295528, 3.85742683,
        4.34060054],
       [3.34582512, 1.09131062, 3.85489853, ..., 3.75465193, 3.89024779,
        4.38870987],
       [3.42233681, 1.13683375, 4.26858055, ..., 3.92658993, 3.97098096,
        4.15364428],
       ...,
       [3.30707131, 1.17512476, 4.36590408, ..., 3.83264537, 3.98403301,
        3.98721178],
       [3.00745493, 1.44786724, 4.17043932, ..., 3.24883756, 3.24900082,
        3.72617328],
       [3.3511931 , 1.28375694, 3.63058044, ..., 3.41062634, 3.84594981,
        4.42362447]])

In [15]:
nb_samples = 100
mse, mae = 0, 0
prediction = 3

for i in range(nb_samples):
    sleep(0.05)
    r = requests.get(url=url_predict, params=params)
    d = r.json()

    # error for last prediction
    rating = d['rating']
    mse += (rating - prediction) ** 2
    mae += np.abs(rating - prediction)
    
    # predict next
    next_user = d['next_user']
    next_item = d['next_item']
    prediction = pred_ratings[next_user, next_item]

print('mse: ', mse/nb_samples)
print('mae: ', mae/nb_samples)

mse:  1.1710260998661617
mae:  0.9289682649843232


# Second method using simularity with biais

In [16]:
def predict_ratings_bias_sub(ratings,sim,phi=(lambda x:x)):

    r_user = (ratings>0).sum(axis=1)
    m_user = np.divide(ratings.sum(axis=1) , r_user, where=(r_user!=0))
    ratings_moyens = np.dot(m_user.reshape(len(m_user),1), np.ones((1,ratings.shape[1])))
   
    wsum_sim = np.abs(phi(sim)).dot(ratings>0)
    pred = ratings_moyens + np.divide(phi(sim).dot(ratings-(ratings>0)*ratings_moyens),wsum_sim, where= wsum_sim!=0)
    
    return np.minimum(5,np.maximum(1,pred))

In [17]:
sim = similarity(ratings)
pred_ratings = predict_ratings_bias_sub(ratings,sim,phi)
pred_ratings[next_user][next_item]

3.448876482337739

In [18]:
from time import sleep

nb_samples = 100
mse, mae = 0, 0

for i in range(nb_samples):
    sleep(0.05)
    r = requests.get(url=url_predict, params=params)
    d = r.json()
    rating = d['rating']
#     print(f'user:{next_user}, item:{next_item}, ratiing:{rating}')
    prediction = pred_ratings[next_user][next_item]
    params['predicted_score'] =  prediction
    
    next_user = d['next_user']
    next_item = d['next_item']
    
    mse += (rating - prediction)**2
    mae += abs(rating - prediction)

print('mse: ', mse/nb_samples)
print('mae: ', mae/nb_samples)

mse:  0.36283001867589126
mae:  0.46155379493214643


# Method 3 with Kears

In [38]:
# For each sample we input the integer identifiers
# of a single user and a single item
user_id_input = Input(shape=[1], name='user')
item_id_input = Input(shape=[1], name='item')

embedding_size = 50
user_embedding = Embedding(output_dim=embedding_size, input_dim=nb_users + 1,
                           input_length=1, name='user_embedding')(user_id_input)

item_embedding = Embedding(output_dim=embedding_size, input_dim=nb_items + 1,
                           input_length=1, name='item_embedding')(item_id_input)

# reshape from shape: (batch_size, input_length, embedding_size)
# to shape: (batch_size, input_length * embedding_size) which is
# equal to shape: (batch_size, embedding_size)
user_vecs = Flatten()(user_embedding)
item_vecs = Flatten()(item_embedding)

y = Dot(axes=1)([user_vecs, item_vecs])

model = Model(inputs=[user_id_input, item_id_input], outputs=y)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
user_embedding (Embedding)      (None, 1, 50)        5050        user[0][0]                       
__________________________________________________________________________________________________
item_embedding (Embedding)      (None, 1, 50)        15050       item[0][0]                       
__________________________________________________________________________________________________
flatten_11

In [39]:
model.compile(optimizer='adamax', loss='mean_squared_error')

In [40]:
# Useful for debugging the output shape of model
initial_train_preds = model.predict([user_history, item_history])
initial_train_preds.shape

(10000, 1)

In [41]:
# Training the model
history = model.fit([df['user'], df['item']], df['rating'],
                    batch_size=64, epochs=100, validation_split=0.1,
                    shuffle=True)

Train on 9000 samples, validate on 1000 samples
Epoch 1/100
9000/9000 [==============================] - 0s 31us/step - loss: 9.3232 - val_loss: 9.6130
Epoch 2/100
9000/9000 [==============================] - 0s 11us/step - loss: 9.2808 - val_loss: 9.5496
Epoch 3/100
9000/9000 [==============================] - 0s 11us/step - loss: 9.1177 - val_loss: 9.2751
Epoch 4/100
9000/9000 [==============================] - 0s 11us/step - loss: 8.6448 - val_loss: 8.5949
Epoch 5/100
9000/9000 [==============================] - 0s 11us/step - loss: 7.7280 - val_loss: 7.4681
Epoch 6/100
9000/9000 [==============================] - 0s 11us/step - loss: 6.4392 - val_loss: 6.0420
Epoch 7/100
9000/9000 [==============================] - 0s 11us/step - loss: 4.9801 - val_loss: 4.5597
Epoch 8/100
9000/9000 [==============================] - 0s 11us/step - loss: 3.6087 - val_loss: 3.2781
Epoch 9/100
9000/9000 [==============================] - 0s 10us/step - loss: 2.4987 - val_loss: 2.2823
Epoch 10/100
900

In [46]:
nb_samples = 100
mse, mae = 0, 0
prediction = 3 # baseline

for i in range(nb_samples):
    sleep(0.05)
    r = requests.get(url=url_predict, params=params)
    d = r.json()

    # error for last prediction
    rating = d['rating']
    
    # predict next
    prediction = model.predict([[next_user], [next_item]])[0][0]
    params['predicted_score'] =  prediction
    
    next_user = d['next_user']
    next_item = d['next_item']

    mse += (rating - prediction) ** 2
    mae += np.abs(rating - prediction)

print('mse: ', mse/nb_samples)
print('mae: ', mae/nb_samples)

mse:  0.12675485875360765
mae:  0.2937440299987793
